In [ ]:
!pip install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt
from sklearn import svm
import numpy as np

broker_address = "ke46e1b0.ala.cn-hangzhou.emqxsl.cn"  # 替换为你的MQTT代理地址
port = 8883
topic = "patient/heartbeat"
username = "test"  # MQTT用户名
password = "1"     # MQTT密码

# 训练简单的SVM模型来分类心跳数据
def train_svm():
    # 正常心跳数据 (90-120)
    X_normal = np.random.randint(90, 121, (50, 1))
    y_normal = np.zeros(50)  # 正常

    # 异常心跳数据
    X_abnormal = np.concatenate([np.random.randint(60, 90, (25, 1)), np.random.randint(121, 150, (25, 1))])
    y_abnormal = np.ones(50)  # 异常

    # 合并数据
    X = np.vstack((X_normal, X_abnormal))
    y = np.concatenate((y_normal, y_abnormal))

    model = svm.SVC(kernel='linear')
    model.fit(X, y)
    return model

model = train_svm()

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to broker")
        client.subscribe(topic)
    else:
        print("Connection failed with code", rc)

def on_message(client, userdata, message):
    heartbeat = int(message.payload.decode("utf-8"))
    print(f"Received heartbeat: {heartbeat}")

    # 预测心跳状态
    prediction = model.predict([[heartbeat]])
    if prediction == 0:
        print("Heartbeat is normal")
    else:
        print("Heartbeat is abnormal")

client = mqtt.Client()
client.username_pw_set(username, password)  # 设置MQTT用户名和密码
client.on_connect = on_connect
client.on_message = on_message

client.connect(broker_address, port)
client.loop_forever()

